In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l1, l2, l1_l2

In [2]:
train = pd.read_pickle('train_clean.pkl')
test = pd.read_pickle('test_clean.pkl')

In [3]:
train.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_210,feature_211,feature_212,feature_213,feature_214,feature_215,feature_216,feature_217,feature_218,target
0,0.554222,1.0,0.0,0.000000,0.398189,0.000000,0.190514,0.774141,0.004995,0.666667,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.0,1
1,0.521013,0.5,0.0,0.000691,0.299222,0.000649,0.000000,0.827157,0.000668,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.0,1
2,0.545612,1.0,0.0,0.000102,0.248790,0.000000,0.571429,0.851531,0.000604,0.000000,...,0.206293,0.022686,0.000562,0.50000,0.116689,0.566809,0.020597,0.020891,1.0,1
3,0.593774,0.5,0.0,0.000000,0.412052,0.001299,0.087886,0.559571,0.002227,0.333333,...,0.026009,0.074658,0.003375,1.00000,0.025595,0.097360,0.020597,0.002580,0.0,1
4,0.278900,0.0,0.0,0.000000,0.160199,0.009740,0.134400,0.841197,0.000700,0.833333,...,0.049540,0.064444,0.003375,0.85715,0.032973,0.131737,0.020597,0.002613,1.0,1


In [4]:
test.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_205,feature_207,feature_208,feature_210,feature_211,feature_212,feature_214,feature_217,feature_218,target
0,0.485838,0.5,0.0,0.000623,0.364916,0.000682,0.000000,0.559775,0.000692,0.000000,...,0.781460,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1
1,0.471255,0.5,0.0,0.000000,0.432846,0.001364,0.089721,0.740107,0.002308,0.333333,...,0.792558,0.013420,0.026187,0.026009,0.074658,0.003963,0.025595,0.002580,0.0,1
2,0.619600,0.5,0.0,0.000000,0.208381,0.002046,0.424221,0.668134,0.000857,0.000000,...,0.781460,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1
3,0.600626,1.0,0.0,0.000018,0.146909,0.015007,0.583363,0.858019,0.001088,0.500000,...,0.789253,0.011287,0.029193,0.176415,0.238134,0.005945,0.043148,0.004473,0.0,1
4,0.642833,1.0,0.0,0.000259,0.177191,0.042974,0.583363,0.691851,0.000956,0.750000,...,0.783863,0.000320,0.216927,0.211176,0.106760,0.001321,0.122571,0.005346,0.0,1


In [5]:
common_cols = set(train.columns).intersection(set(test.columns))

In [6]:
train = train[common_cols]

test = test[common_cols]

C:\Users\franz\AppData\Local\Temp\ipykernel_21272\982039621.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  train = train[common_cols]
C:\Users\franz\AppData\Local\Temp\ipykernel_21272\982039621.py:3: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  test = test[common_cols]


In [7]:
X_train = train.drop(columns=['target'])
y_train = train['target']

In [8]:
X_test = test.drop(columns=['target'])
y_test = train['target']

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [10]:
rus = RandomUnderSampler(sampling_strategy = 'auto')

In [11]:
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

In [12]:
modelo = Sequential()

modelo.add(Dense(128, activation = 'relu', kernel_regularizer = l2(0.01), input_shape = (X_train.shape[1],)))
#modelo.add(Dense(128, activation = 'tanh'))
#modelo.add(Dropout(0.2))
modelo.add(Dense(64, activation = 'relu'))
modelo.add(Dense(32, activation = 'relu'))
modelo.add(Dense(16, activation = 'relu'))
modelo.add(Dropout(0.2))

modelo.add(Dense(1, activation = 'tanh'))

In [13]:
lr = 0.001
optim = Adam(learning_rate = lr)

In [14]:
modelo.compile(optimizer = optim, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [15]:
history = modelo.fit(X_train_resampled, y_train_resampled, epochs = 500, batch_size = 50, validation_data = (X_val, y_val), verbose = 1)

Epoch 1/500
2128/2128 [==============================] - 18s 8ms/step - loss: 0.8352 - accuracy: 0.5122 - val_loss: 0.7015 - val_accuracy: 0.5328
Epoch 2/500
2128/2128 [==============================] - 18s 8ms/step - loss: 0.7035 - accuracy: 0.5145 - val_loss: 0.7469 - val_accuracy: 0.1515
Epoch 3/500
2128/2128 [==============================] - 18s 8ms/step - loss: 0.6946 - accuracy: 0.5146 - val_loss: 0.7134 - val_accuracy: 0.1590
Epoch 4/500
2128/2128 [==============================] - 19s 9ms/step - loss: 0.6932 - accuracy: 0.5148 - val_loss: 0.6787 - val_accuracy: 0.6611
Epoch 5/500
2128/2128 [==============================] - 19s 9ms/step - loss: 0.6927 - accuracy: 0.5194 - val_loss: 0.6814 - val_accuracy: 0.6129
Epoch 6/500
2128/2128 [==============================] - 18s 9ms/step - loss: 0.6913 - accuracy: 0.5355 - val_loss: 0.6952 - val_accuracy: 0.5389
Epoch 7/500
2128/2128 [==============================] - 20s 9ms/step - loss: 0.6887 - accuracy: 0.5472 - val_loss: 0.6176 -

KeyboardInterrupt: 